In [1]:
import os
import re
import json
import pandas as pd
from collections import Counter
import spacy
from spacy.tokens import Doc

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn

sentiment_analyzer = SentimentIntensityAnalyzer()
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)
Doc.set_extension('polarity_scores', getter=polarity_scores)
nlp = spacy.load("en_core_web_lg")    

from tqdm import tqdm


In [2]:
convTimelineCell = 42
minLines = 50
stripPersonnas = True
main = ['jerry', 'summer', 'beth', 'rick', 'morty', 'pickle rick', 'unity']
keysToRemove = ['fullText', 'pron', 'lines']
replace = {
    'rick sanches': 'rick',
    'drunk rick': 'rick',
    'morty smith': 'morty',
    'gromfomite': 'gromflomite',
    'other gromflomite': 'gromflomite',
    'gromflomite captain': 'gromflomite',
    'gromflomite worker': 'gromflomite',
    'gromflomite guard': 'gromflomite',
    'gromflomite office employee': 'gromflomite',
    'rick salseman': 'rick salesman',
    'armed ricks': 'armed rick',
    'all religious mortys': 'religious morty',
    'little tommy': 'tommy',
    'meeseek': 'meeseeks',
    'all the meeseeks': 'meeseeks',
    'all other meeseeks': 'meeseeks',
    'mailmen': 'mailman',
    'gerry': 'jerry'
    
}
ignore = [
    'a mexican armada shows up. with weapons made from two', 
    'pa system', 
    'all classmates except morty',
    'another voice',
    'sexualized s',
    'both']

#main = ['beth']
data = {
    'characters': {},
}

In [3]:

 
def do_nlp(character, line, file, section):
    non_actions = re.sub(r'\[.*\]|\(.*\)', '', line)
    non_actions = re.sub(r'[!\.\?\;\"\-\*]', ' ', non_actions)
    non_actions = re.sub(r'([a-z])\1{2,}',r'\1', non_actions)

    text = '';
    
    doc = nlp(non_actions)
    
    for token in doc:
        if (token.lemma_ != '-PRON-'):        
            text = text + token.lemma_ + ' '        
    
    character['fullText'] += non_actions    
    score = doc._.polarity_scores['compound']

    actualVal =  character['sentiment'][section] if character['sentiment'][section] != False else 0
    character['sentiment'][section] = (actualVal + score) / 2
    
    actualVal =  character['episodes'][file]['sentiment'][section] if character['episodes'][file]['sentiment'][section] != False else 0
    character['episodes'][file]['sentiment'][section] = (actualVal + score) / 2
    
    if abs(score)>.2:
        character['episodes'][file]['sentimentLine'][section] += non_actions;

    
#     s = flair.data.Sentence(line)
#     flair_sentiment.predict(s)
    
#     try:
#         score = s.labels[0].to_dict() 
#         score = score['confidence'] if score['value'] == 'POSITIVE' else -score['confidence']
        
#         actualVal =  character['sentiment'][section] if character['sentiment'][section] != False else 0
#         character['sentiment'][section] = (actualVal + score) / 2
        
#         actualVal =  character['episodes'][file]['sentiment'][section] if character['episodes'][file]['sentiment'][section] != False else 0
#         character['episodes'][file]['sentiment'][section] = (actualVal + score) / 2
#     except:
#         pass
#         print('Error')

In [4]:
def getCharacter(characterName):
    characters = data['characters']
    try:
        character = characters[characterName]
    except KeyError:
        character = characters[characterName] = {
            'linesCount': 0, 
            'lines': [], 
            'time': [0] * convTimelineCell, 
            
            'episodes': {},
            'pron': {},
            'fullText': ''
        }
        if characterName in main:
            character['sentiment'] = ([False] * convTimelineCell)
            character['sentimentLine'] = ([''] * convTimelineCell)

    return character

def parseLine(characterName, line, file, section):
    character = getCharacter(characterName)    
    character['linesCount'] += 1    
    episodes = character['episodes']
    
    try: 
        episode = episodes[file]
    except:
        episode = episodes[file] = {
            'linesCount': 0,
            'time': [0]*convTimelineCell,
            
        }
        if characterName in main:
            episode['sentiment'] = ([False] * convTimelineCell)
            episode['sentimentLine'] = ([''] * convTimelineCell)
        
    character['time'][section] += 1
    episode['time'][section] += 1
    episode['linesCount'] += 1
    
    if characterName in main:
        do_nlp(character, line, file, section)

In [5]:
files = os.listdir("./parsed")
for filename in files:
    print('Processing: ', filename)
    lines = 0
    with open("./parsed/" + filename, "r") as f:             
        for line in f:            
            lines += 1
            
    with open("./parsed/" + filename, "r") as f:                    
        index = 0
        for line in tqdm(f, total=lines): 
            if stripPersonnas == True:
                find = re.search(r"^([a-zA-Z][a-zA-Z \.]+).*\:(.*)", line) 
            else:
                find = re.search(r"^([a-z][a-z0-9 \(\)]+):(.*)", line)

            if find:
                name = find.group(1).strip()
                text = find.group(2).strip()
                if name not in ignore:
                    try:
                        actualName = replace[name]
                    except:
                        actualName = name
                    
                    parseLine(actualName, text, filename[6:len(filename)-4], int(index * convTimelineCell / lines))
            index += 1

  8%|▊         | 19/245 [00:00<00:01, 185.56it/s]

Processing:  011 - Ricksy Business.txt


 10%|█         | 27/262 [00:00<00:00, 260.45it/s]

Processing:  021 - The Wedding Squanchers.txt


  7%|▋         | 23/319 [00:00<00:01, 228.02it/s]

...

  6%|▌         | 21/376 [00:00<00:01, 201.13it/s]

Processing:  023 - Rickmancing the Stone.txt


  4%|▍         | 18/411 [00:00<00:02, 177.40it/s]

Processing:  012 - A Rickle in Time.txt


100%|██████████| 411/411 [00:01<00:00, 312.72it/s]


In [6]:
# toRemove = []
# characters = data['characters']
# data['allCharacterNames'] = set(characters.keys())
# for key in characters.keys():
#     count = characters[key]['linesCount']
#     if count < minLines:
#         toRemove.append(key)
        
# for keyToRemove in toRemove:
#     del characters[keyToRemove]
    
# print(characters.keys())

In [7]:
for key in main:
    try:
        character = data['characters'][key]
        full_text = character['fullText']
        doc = nlp(full_text)
        nouns = [token.text for token in doc if token.is_stop != True and token.is_punct != True and token.pos_ == "NOUN"]
        word_freq = Counter(nouns)
        character['common_words'] = word_freq.most_common(50)    
        print('-'.join(['-']*10))
        print(key)
        print(character['common_words'])
    except KeyError:
        pass

for key in data['characters'].keys():
    
    character = data['characters'][key]
    for keyToRemove in keysToRemove:
        try:
            del character[keyToRemove]
        except KeyError:
            pass
    episodes = character['episodes']
    
    for episode in episodes.keys():
        print(episode)
        for keyToRemove in keysToRemove:
            try:
                del character['episodes'][episode][keyToRemove]
            except KeyError:
                pass

    
    
    

-------------------
jerry
[('morty', 24), ('planet', 23), ('guys', 20), ('son', 19), ('thing', 18), ('summer', 18), ('time', 18), ('family', 16), ('father', 16), ('rick', 13), ('jerry', 12), ('house', 11), ('science', 11), ('way', 10), ('life', 8), ('honey', 8), ('wife', 8), ('people', 7), ('fun', 7), ('hell', 7), ('gary', 7), ('guy', 7), ('christmas', 7), ('kids', 7), ('things', 7), ('pluto', 7), ('idea', 6), ('school', 6), ('friend', 6), ('head', 6), ('dad', 6), ('man', 6), ('apples', 6), ('deer', 6), ('tv', 5), ('device', 5), ('job', 5), ('love', 5), ('milk', 5), ('home', 5), ('face', 5), ('kind', 5), ('dog', 5), ('mom', 5), ('cloud', 4), ('atlas', 4), ('tru', 4), ('surgeon', 4), ('game', 4), ('movie', 4)]
-------------------
summer
[('dad', 39), ('grandpa', 39), ('morty', 25), ('rick', 19), ('mom', 12), ('guys', 11), ('people', 8), ('head', 7), ('time', 7), ('unity', 7), ('drum', 7), ('pants', 6), ('school', 6), ('place', 6), ('parents', 5), ('thing', 5), ('family', 5), ('god', 5),

In [8]:
with open('data.json', 'w') as fp:
    json.dump(data, fp)